# Module 6

## Basic LangChain invoke and chain

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

# Uses ChatBedrockConverse hidden under init_chat_model
model = init_chat_model(
    model="us.amazon.nova-lite-v1:0", 
    model_provider="bedrock_converse"
)

messages= [
    SystemMessage("Translate the following from English into French"),
    HumanMessage("hi!"),
]

response= model.invoke(messages)

print(response)

content='Bonjour! Comment ça va? \n\n(Translation: Hi! How are you?)' additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '99852a5a-4ed9-4f2d-8708-3ee019191790', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 28 Sep 2025 09:40:51 GMT', 'content-type': 'application/json', 'content-length': '261', 'connection': 'keep-alive', 'x-amzn-requestid': '99852a5a-4ed9-4f2d-8708-3ee019191790'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [505]}, 'model_name': 'us.amazon.nova-lite-v1:0'} id='run--40dfaabd-35f5-47e8-8664-362f34342657-0' usage_metadata={'input_tokens': 9, 'output_tokens': 17, 'total_tokens': 26, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}


### Chaining the model with the StrOutputParser to render the output

In [2]:
chain = model | StrOutputParser()

print(chain.invoke(messages))

Bonjour! 

If you have any specific sentences or phrases you'd like translated from English to French, feel free to share them!


## LangChain PromptTemplates

In [17]:
from langchain_core.prompts import ChatPromptTemplate

# Set the system prompt template using language as a variable
system_template = "Translate the following from English into {language}"

# Set the final prompt combining system prompt and the user's text
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template), 
    ("user", "{text}")
])

# Chain the prompt_template, model and renderer
chain = prompt_template | model | StrOutputParser()

# Invoke the chain passing in the arguments for the prompt template
response = chain.invoke({
    "language": "Italian", 
    "text": "hi!"
})

print(response)

Ciao! 

Se hai bisogno di aiuto con la traduzione di un testo più lungo o di una frase specifica, sono qui per aiutarti.


## LangChain conversation history in DynamoDB

### Setting up the DynamoDB table

In [ ]:
import boto3
table_name = "langchain-conversation-history"

dynamodb = boto3.client('dynamodb')
response = dynamodb.create_table(
    TableName=table_name,
    BillingMode='PAY_PER_REQUEST',
    AttributeDefinitions=[
        {
            'AttributeName': 'SessionId',
            'AttributeType': 'S'
        }
    ],
    KeySchema=[
        {
            'AttributeName': 'userId',
            'KeyType': 'HASH'
        }
    ]
)

# Wait for table to be active
waiter = dynamodb.get_waiter('table_exists')
waiter.wait(TableName=table_name)

ttl_response = dynamodb.update_time_to_live(
    TableName=table_name,
    TimeToLiveSpecification={
        'AttributeName': 'expireAt',
        'Enabled': True
    }
)

### Testing with DynamoDB

In [ ]:
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory

history = DynamoDBChatMessageHistory(
    table_name="langchain-conversation-history", 
    session_id="1"
)

history.add_user_message("hi!")
history.add_ai_message("whats up?")

### Setting up the model, prompt template and chain

In [ ]:
from langchain_core.prompts import MessagesPlaceholder

# Typical model definition
model = init_chat_model(
    model="us.amazon.nova-lite-v1:0", 
    model_provider="bedrock_converse"
)

# Creates a prompt template that will have the system prompt, the
# history from the variable history and human question
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# Creates a chain by starting with the prompt_template, the model
# and parsing the output
chain = prompt_template | model | StrOutputParser()

### Adding the message history in the chain

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    # Existing LangChain processing pipeline
    chain, 
    
    # Factory function that sends session_id as a parameter to the 
    # DynamoDBChatMessageHistory() function which creates a new history 
    # per session
    lambda session_id: DynamoDBChatMessageHistory(    
        table_name="langchain-conversation-history", 
        session_id=session_id, # From the parameter
        ttl=120
    ),
    
    # Tells the wrapper that user input comes from the question field
    input_messages_key="question", 
    
    # Maps to the MessagesPlaceholder variable name in the prompt template
    history_messages_key="history" 
)

### Setting the session_id and invoking the chain

In [11]:
# Setting the session_id in the config so it can be passed
config = { "configurable": { "session_id": "24" } }

# Invoking the chain twice
chain_with_history.invoke({"question": "Hi! I'm Alice."}, config=config)
chain_with_history.invoke({"question": "What's my name?"}, config=config)


'Your name is Alice. If you have any other questions or need assistance with something else, feel free to ask!'